This workflow is written to pull meetings data from a table, format all the meeting information,
and remotely insert it into BMLT MySQL database set up as new.  For testing, I manually entered
several meetings and queried them from the connections set up below in order to get the process
right. 

In order to enable remote SQL connections, you need to add your IP address under cPanel/Remote MySQL under the Advanced cPanel section. You can run an ipconfig command in command shell or Google what is my IP to get that - please note that your IP address might get changed each time you connect to a router, so you might want to use a wildcard % (so if your home is 222.333.23.213 you might put 222.333.23.%).  

The data you have might be in an Excel or CSV table, if so see the Pandas functions read_csv or
read_excel in order to pull into a DataFrame.  In my case, I had everything as a JSON saved on 
our website as a JavaScript file. I pull this, convert it, and insert it into the BMLT server.

Use freely, hopefully some portion of this might be helpful - if only pulling data remotely. For 
assistance, please email aleczoeller at gmail dot com, or call at Nine 1 two, 358 six 5 six 4.

In [6]:
import MySQLdb
import pandas as pd
import numpy as np
import json
import requests
import googlemaps
from pandas.io.json import json_normalize
import sqlalchemy as sa

import base64
from dotenv import load_dotenv

In [ ]:
#Load environment variables from a .env file.  Update with your specific passwords/keys/sources
load_dotenv()
GOOGLE_API_KEY = base64.b64decode(os.environ['GOOGLE_API_KEY']).decode('ascii')
SITE_ADDRESS = base64.b64decode(os.environ['SEMANTIC_SITE_URL']).decode('ascii')

In [ ]:
#Create connection with cPanel MYSQL database. You have to whitelist your IP address before connecting!
engine = sa.create_engine('mysql+pymysql://<cpanel_username>:<password>@<site_ip_address>/<bmlt db name>')

In [ ]:
#Alternate connection with cPanel MYSQL database. You have to whitelist your IP address before connecting!
conn = MySQLdb.connect(host='<site_ip_address', user='<cpanel_username',
                      passwd='password', db='<bmlt db name')

In [ ]:
#Pull existing table information.  This isn't required, but if you enter a sample meeting can be helpful
#in order to visualize target table schema
query1 = "SELECT * FROM `na_comdef_meetings_main` WHERE 1"
query2 = "SELECT * FROM `na_comdef_meetings_data` WHERE 1"
query3 = "SELECT * FROM `na_comdef_service_bodies` WHERE 1"
df = pd.read_sql(con=conn, sql=query1)
df

In [ ]:
formats = "SELECT * FROM `na_comdef_formats` WHERE 1"
formats = pd.read_sql(con=conn, sql=formats)

In [ ]:
#Again, not required to check existing tables, but useful in creating this workflow
df2 = pd.read_sql(con=conn, sql=query2)
df3 = pd.read_sql(con=conn, sql=query3)

In [ ]:
conn.close()

In [ ]:
#My own approach to pulling currently organized meeting information. If table is saved locally,
#see pandas read_csv or read_excel methods
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
mdata = requests.get(SITE_ADDRESS, headers=headers)

In [ ]:
mdata = mdata.text[28:].replace('\n', '').replace('\/', '/')
mdata = json.loads(mdata)

In [ ]:
#Meetings are in json format from GET request. Convert to Pandas DataFrame object, and check output
meetings = json_normalize(mdata['features'])

In [ ]:
#Create dataframes to insert data into database. Use same schema from database
init_id = 2 #Add one value to the last meeting entered. If none have been entered, use 1 here.
meetings_data = pd.DataFrame({}, columns=df2.columns)
meetings_main = pd.DataFrame({}, columns = df.columns)
day_dict = {'M':1,'Tu':2, 'W':3, 'Th':4, 'F':5, 'Sa':6, 'Su':0}
#Get service bodies (manually entered into BMLT Root Server) from their respective table, as dictionary
sbs = dict([[row['name_string'].split(' ')[0], row['id_bigint']] for _, row 
                in df3.iterrows()])
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)
for _, meeting in meetings.iterrows():
    
    #Get formats for meetings in name, from NAWS codes queried above from MySQL database
    a = meeting['properties.Group']
    fs = []
    if '(' in a:
        b = a.split('(')[1].split(')')[0]
        if 'Men' in b:
            fs.append('15')
        if 'Women' in b:
            fs.append('32')
        if 'SS' in b:
            fs.append('27')
        if 'TS' in b:
            fs.append('30')
        if 'NS' in b:
            fs.append('37')
        if 'C' in b and not 'CAN' in b and not 'WC' in b:
            fs.append('4')
        if 'BT' in b:
            fs.append('3')
        if not '4' in fs:
            fs += ['17','33']   
    else:
        fs = ['17','33']
    
    
    #Get formatted address and zip codes (not currently tagged) for meeting    
    address = meeting['properties.Address'].replace('URS Club ', '')
    result = gmaps.geocode(address)
    for a in result[0]['address_components']:
        if a['types'][0] == 'street_number':
            street1 = a['long_name']
        if a['types'][0] == 'route':
            street2 = a['long_name']
        if a['types'][0] == 'locality':
            city = a['long_name']
        if a['types'][0] == 'administrative_area_level_1':
            state = a['short_name']
        if a['types'][0] == 'postal_code':
            zipcode = a['long_name']
    street = ' '.join([street1, street2])
    #Get updated format name
    mgroup = meeting['properties.Group'].split('(')[0]
    mgroup = mgroup[:-1] if mgroup[-1] == ' ' else mgroup
    formats_insert = ','.join(fs)
    #Get name of building, if any
    if meeting['properties.Address'][0].isdigit() and not '1st Baptist' in meeting['properties.Address']:
        location1 = ''
    elif 'URS' in meeting['properties.Address']:
        location1 = 'URS Club'
    else:
        location1 = meeting['properties.Address'].split(',')[0]
        
    #Meeting on separate rows will be split into separate rows/entries
    if ',' in meeting['properties.Day']:
        for day in meeting['properties.Day'].split(','):
            a = [init_id, 'meeting_name', 'Meeting Name', 'en', 0.0, mgroup, np.nan, None]
            b = [init_id, 'location_text', 'Location Name', 'en', 0.0, location1, np.nan, None]
            c = [init_id, 'location_street', 'Street Address', 'en', 0.0, street, np.nan, None]
            d = [init_id, 'location_municipality', 'Town', 'en', 0.0, city, np.nan, None]
            e = [init_id, 'location_province', 'State', 'en', 0.0, state, np.nan, None]
            f = [init_id, 'location_postal_code_1', 'Zip Code', 'en', 0.0, int(zipcode), np.nan, None]
            g = [init_id, 'location_nation', 'Nation', 'en', 0.0, 'USA', np.nan, None]
            for newline in [a,b,c,d,e,f,g]:
                meetings_data.loc[len(meetings_data), :] = newline
            #Insert meeting info as single line into second table
            t = meeting['properties.Time']
            if t[:4].lower() == 'noon':
                startt = '12:00:00'
            elif t[2] == '3':
                hour = str(12 + int(t[0])) if int(t[:1].replace(':', '')) < 10 else t[:1]
                startt = '{}:30:00'.format(hour)
            elif t[3]== '3':  #Irregular times here
                hour = str(12+int(t[0])) if int(t[:1].replace(':', '')) < 10 else t[:1]
                minute = t.split(' ')[0].split(':')[1]
                startt = '{0}:{1}:00'.format(hour, minute)
            else:
                hour = str(12+int(t[0])) if int(t[:1].replace(':', '')) < 10 else t[:1]
                startt = '{}:00:00'.format(hour)
            serviceb = sbs[meeting['properties.Area']] if meeting['properties.Area'] in sbs.keys() else 6
            newline = [init_id, None, None, serviceb, day_dict[day.replace(' ', '')], startt, '01:00:00', 
                       formats_insert, 'en', meeting['geometry.coordinates'][0], meeting['geometry.coordinates'][1],
                      1, 'aleczoeller@gmail.com']
            meetings_main.loc[len(meetings_main), :] = newline
            init_id += 1
    #Meetings that take place on one day only get inserted once        
    else:
        a = [init_id, 'meeting_name', 'Meeting Name', 'en', 0.0, mgroup, np.nan, None]
        b = [init_id, 'location_text', 'Location Name', 'en', 0.0, location1, np.nan, None]
        c = [init_id, 'location_street', 'Street Address', 'en', 0.0, street, np.nan, None]
        d = [init_id, 'location_municipality', 'Town', 'en', 0.0, city, np.nan, None]
        e = [init_id, 'location_province', 'Town', 'en', 0.0, state, np.nan, None]
        f = [init_id, 'location_postal_code_1', 'Zip Code', 'en', 0.0, int(zipcode), np.nan, None]
        g = [init_id, 'location_nation', 'Nation', 'en', 0.0, 'USA', np.nan, None]
        #Each attribute gets its own line in the MySQL db
        for newline in [a,b,c,d,e,f,g]:
            meetings_data.loc[len(meetings_data), :] = newline
        #Insert meeting info as single line into second table
        t = meeting['properties.Time']
        if t[:4].lower() == 'noon':
            startt = '12:00:00'
        elif t[2] == '3':
            hour = str(12 + int(t[0])) if int(t[:1].replace(':', '')) < 10 else t[:1]
            startt = '{}:30:00'.format(hour)
        elif t[3]== '3':  #Irregular times here
            hour = str(12+int(t[0])) if int(t[:1].replace(':', '')) < 10 else t[:1]
            minute = t.split(' ')[0].split(':')[1]
            startt = '{0}:{1}:00'.format(hour, minute)
        else:
            hour = str(12+int(t[0])) if int(t[:1].replace(':', '')) < 10 else t[:1]
            startt = '{}:00:00'.format(hour)
        serviceb = sbs[meeting['properties.Area']] if meeting['properties.Area'] in sbs.keys() else 6
        newline = [init_id, None, None, serviceb, day_dict[day.replace(' ', '')], startt, '01:00:00', 
                   formats_insert,'en', meeting['geometry.coordinates'][0], meeting['geometry.coordinates'][1],
                  1, 'aleczoeller@gmail.com']
        meetings_main.loc[len(meetings_main), :] = newline
        init_id += 1


In [ ]:
#Clean formats for insertion into db
meetings_main.apply(lambda x: x['formats'].replace("'", ""), axis=1)

#Insert meetings_main dataframe directly to database
meetings_main.to_sql('na_comdef_meetings_main', con=engine, if_exists='append', index=False)

In [ ]:
#Insert individual meetings data lines to db
meetings_data.to_sql('na_comdef_meetings_data', con=engine, if_exists='append', index=False)
#DONE!!!  Check the results for both tables in the BMLT main_server page.